In [2]:
library(here)
library(here)
library(readr, warn.conflicts = FALSE)
library(arrow, warn.conflicts = FALSE)
library(magrittr, warn.conflicts = FALSE)
library(stringr, warn.conflicts = FALSE)
library(dplyr, warn.conflicts = FALSE)
library(rlang, warn.conflicts = FALSE)
library(data.table, warn.conflicts = FALSE)
library(lubridate, warn.conflicts = FALSE)
library(tidyr, warn.conflicts = FALSE)
library(truveta.study)
library(dbplyr)
library(lubridate)

here() starts at /home/jovyan/studyrepositories/hypertensioncohorts-is5fyzbqztbebenalfbuo4boem

Loading required package: DBI


Attaching package: ‘dbplyr’


The following objects are masked from ‘package:dplyr’:

    ident, sql




In [3]:
# build connections 
con <- create_connection()
study <- get_study(con, title = "Hypertension Cohort")
population <- get_population(con, study, title = "Study Cohort")
snapshot <- get_latest_snapshot(con, population, finished = TRUE)

To sign in to Truveta, use a web browser to open the page https://login.truveta.com/activate?user_code=VLSF-TMJW.

Signed in successfully.



In [3]:
get_data_dictionary(con,snapshot) %>% group_by(table) %>% summarise(cnt = n())

table,cnt
<chr>,<int>
Claim,13
ClaimLine,6
ClaimLineCodes,4
Concept,7
ConceptLink,2
Condition,14
ConditionCodes,4
DeviceUse,14
Encounter,11


In [24]:
sql <- "select s.AttributeConceptId, c.ConceptName, count(distinct PersonId) as PatientCount , count(distinct s.Id) as RecordCount
from SocialDeterminantsOfHealth s join Concept c on (s.AttributeConceptId = c.ConceptId)
group by AttributeConceptId, ConceptName
order by 3;"

attributes_name <- load_sql_table(con,snapshot,query= sql) %>% collect()

In [6]:
attributes_name

AttributeConceptId,ConceptName,PatientCount,RecordCount
<int>,<chr>,<int>,<int>
3058869,EvictionAge,1049504,1080020
3058868,CurrAddrLenOfRes,1049504,1081606
3058861,HighestEduHousehold,1057265,1061551
3058866,RaAMmbrCnt,1057265,1076469
2506717,HouseholdMotorizedPropertyRegistrationsCount,1057265,1067105
3058867,ShortLenofResBcEviction,1057265,1060652
3058865,CurrAddrMedianIncome,1057265,1062669
2506719,IndividualMotorizedPropertyRegistrationsCount,1057265,1059303
3058449,CurrAddrDwellType,1057265,1060047


In [25]:
# number of patients who have at least one SDOH element
sql <- "select 'All Patients' as Type, count(distinct PersonId) as cnt
from SocialDeterminantsOfHealth
union
(select 'NonNull Patient' as Type, count(distinct PersonId) as cnt
from SocialDeterminantsOfHealth
where NormalizedValueNumeric is not NULL and EffectiveStartDateTime is not NULL);"

counts <-load_sql_table(con,snapshot,sql)%>% collect()

In [8]:
counts

Type,cnt
<chr>,<int>
All Patients,1067200
NonNull Patient,1048615


In [8]:
concept <- read_parquet("data/Concept.parquet")

In [9]:
# distribution for the normalized value concept id
sql <- "select s.AttributeConceptId, NormalizedValueConceptId, c.ConceptName, count(distinct PersonId) as PatientCount, 
count(distinct s.Id) as RecordCount
from SocialDeterminantsOfHealth s join Concept c on (s.NormalizedValueConceptId = c.ConceptId)
group by s.AttributeConceptId, NormalizedValueConceptId, ConceptName
order by 1;"

valueconcept <- load_sql_table(con,snapshot,query = sql) %>% collect() %>% select(AttributeConceptId, NormalizedValueConceptId, 
                                                                                  NormalizedValueConceptName = ConceptName,
                                                                                  PatientCount,RecordCount)%>% 
merge(., concept, by.x = "AttributeConceptId", by.y = "ConceptId",all.x = TRUE)%>%
select(AttributeConceptId, NormalizedValueConceptId, 
       NormalizedValueConceptName, AttributeName = ConceptName,PatientCount,RecordCount)
valueconcept

AttributeConceptId,NormalizedValueConceptId,NormalizedValueConceptName,AttributeName,PatientCount,RecordCount
<int>,<int>,<chr>,<chr>,<int>,<int>
2506706,1067558,Unknown,CurrentAddressOwnOrRent,264873,268381
2506706,2506674,Own,CurrentAddressOwnOrRent,626987,632762
2506706,2506675,Rent,CurrentAddressOwnOrRent,155887,158036
2506706,1067561,No Information,CurrentAddressOwnOrRent,2978,3030
2506707,2506677,25 - 100 miles,DistanceToCloseTies,17075,17349
2506707,2506676,100 - 500 miles,DistanceToCloseTies,16925,17210
2506707,2506678,Less than 25 miles,DistanceToCloseTies,962963,972091
2506707,2506679,Over 500 miles,DistanceToCloseTies,18121,18439
2506707,1067561,No Information,DistanceToCloseTies,34626,35241


In [4]:
#get numeric attributes distribution
sql <- "select s.AttributeConceptId, c.ConceptName,count(distinct s.Id) as NonMissingCount,
(count(*) - count(distinct Id)) as MissingValuesCount,
avg(NormalizedValueNumeric) as Average, min(NormalizedValueNumeric) as Min,
max(NormalizedValueNumeric) as Max
from
SocialDeterminantsOfHealth s join Concept c on (s.AttributeConceptId = c.ConceptId)
where AttributeConceptId not in (3058440,3058449,2506706,3058860,3058861,3058867)
group by AttributeConceptId,ConceptName
order by 1;"

numeric_attr <- load_sql_table(con,snapshot, query =sql) %>% collect()
numeric_attr

AttributeConceptId,ConceptName,NonMissingCount,MissingValuesCount,Average,Min,Max
<int>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>
2506701,AddressChangeCountLast12Months,1080587,0,0.122356,0.0000,25.0000
2506702,AddressChangeCountLast1Month,1077788,0,0.003027,0.0000,4.0000
2506703,AddressChangeCountLast3Months,1079667,0,0.021208,0.0000,6.0000
2506704,AddressChangeCountLast60Months,1081181,0,0.698655,0.0000,100.0000
2506705,AddressChangeCountLast6Months,1080773,0,0.059176,0.0000,10.0000
2506707,DistanceToCloseTies,1078970,0,0.067757,-1.0000,3.0000
2506712,FirstDegreeRelativesCount,1092659,0,9.808893,0.0000,100.0000
2506715,HouseholdMembersCount,1091668,0,2.944950,1.0000,255.0000
2506717,HouseholdMotorizedPropertyRegistrationsCount,1067105,0,2.550157,0.0000,228.0000


In [17]:
# get distirbution of categorical attributes

sql <-  "select s.AttributeConceptId, s.NormalizedValueConceptId, c.ConceptName as 'AttributeName',
count(distinct s.Id) as n from
SocialDeterminantsOfHealth s join Concept c on (s.AttributeConceptId = c.ConceptId)
where AttributeConceptId in (3058440,3058449,2506706,3058860,3058861,3058867)
group by AttributeConceptId,NormalizedValueConceptId,ConceptName;"
category_attr <- load_sql_table(con,snapshot,query = sql)%>% collect() %>% merge(.,concept,by.x = "NormalizedValueConceptId",
                                                                                 by.y = "ConceptId", all.x = TRUE)%>%
select(AttributeConceptId, NormalizedValueConceptId,AttributeName, ValueName = ConceptName,n)%>%
arrange(AttributeConceptId, NormalizedValueConceptId)
category_attr

AttributeConceptId,NormalizedValueConceptId,AttributeName,ValueName,n
<int>,<dbl>,<chr>,<chr>,<int>
2506706,1067558,CurrentAddressOwnOrRent,Unknown,268381
2506706,1067561,CurrentAddressOwnOrRent,No Information,3030
2506706,2506674,CurrentAddressOwnOrRent,Own,632762
2506706,2506675,CurrentAddressOwnOrRent,Rent,158036
2506706,NA,CurrentAddressOwnOrRent,NA,18637
3058440,3058441,CollegePastPresent,No college on record,801526
3058440,3058442,CollegePastPresent,Currently attending,4760
3058440,3058443,CollegePastPresent,Attended but not currently,233640
3058440,NA,CollegePastPresent,NA,17827


In [18]:
# number of patietns with SDOH data element in their absolute year time
index_date <- read_parquet("data/index_date.parquet")
head(index_date)


sql <- "select year(s.EffectiveStartDateTime) as Year,s.Id as SDOHId,sr.*
from
SocialDeterminantsOfHealth s join (
Select PersonId, RecordedDateTime as 'IndexDate', AbsoluteEffectiveDateTime as 'AbsoluteIndexYM'
from(
select PersonId, row_number() over (partition by PersonId order by RecordedDateTime) as 'rownum', 
RecordedDateTime, AbsoluteEffectiveDateTime
from SearchResult_hypertension
group by PersonId, RecordedDateTime, AbsoluteEffectiveDateTime)  as t
where t.rownum =1) as sr on (s.PersonId = sr.PersonId)
order by 1;"


idx_record <- load_sql_table(con,snapshot,query= sql) %>% collect()
head(idx_record,10)

PersonId,IndexDate,AbsoluteIndexYM,ShiftDays
<chr>,<dttm>,<chr>,<int>
00000608-8ba0-385f-c781-4ac002edbff8,2018-03-05,2019-04,392
000009b2-284d-b26b-3e1e-e97dc61cc275,2018-02-22,2019-04,403
000018f2-ff85-1ebe-ed48-cef0e95b650f,2017-02-25,2019-02,706
00002dd6-b955-6eea-4fb9-b3605de42dff,2017-05-11,2019-03,659
00003074-3a64-0fd9-d250-930e7cf7ddaa,2017-11-20,2019-01,407
000037e4-3401-af14-5c44-15dc735b500b,2017-07-06,2019-06,695


Year,SDOHId,PersonId,IndexDate,AbsoluteIndexYM
<dbl>,<chr>,<chr>,<dttm>,<chr>
NA,393f792a-a3f0-b3ad-e9db-55351c3061c8,2ea78d83-3207-f00b-9903-5866286bf7a6,2017-05-08,2019-03
NA,c48b5039-4669-c99c-fbdb-bd3047808874,2ea78d83-3207-f00b-9903-5866286bf7a6,2017-05-08,2019-03
NA,952dddec-cb2d-c479-067d-ddf60af75e67,904c2761-714b-adbe-e6e1-4dec29a1b78b,2017-10-28,2019-01
NA,1e3fc080-d65e-3fc0-fd77-9144ff0fb444,db9ef85e-e350-a564-f59f-235cb2735415,2017-12-20,2019-05
NA,f70d73ec-10df-9bdc-0386-964bd1eaed53,2ea78d83-3207-f00b-9903-5866286bf7a6,2017-05-08,2019-03
NA,823a7e73-35ab-4fd6-056b-18d8f0a4769f,2ea78d83-3207-f00b-9903-5866286bf7a6,2017-05-08,2019-03
NA,d4411081-2a20-cb0c-d184-aea57bd217f1,904c2761-714b-adbe-e6e1-4dec29a1b78b,2017-10-28,2019-01
NA,b5bc1f8d-88ef-6ec5-fd69-445428fbd63b,db9ef85e-e350-a564-f59f-235cb2735415,2017-12-20,2019-05
NA,890fa8e3-5ecf-42dc-0d5c-d21ebdd43269,2ea78d83-3207-f00b-9903-5866286bf7a6,2017-05-08,2019-03


In [ ]:
tables <- list(category_attr,numeric_attr,attributes_name,counts,idx_record)
saveRDS(tables, "data/tables.rds")
#read it in next time
#tables <- readRDS("data/tables.rds")

In [21]:
idx_record %>% group_by(AbsoluteIndexYM) %>% summarise(cnt = n()) 

AbsoluteIndexYM,cnt
<chr>,<int>
2019-01,6745886
2019-02,3997206
2019-03,3466142
2019-04,2827390
2019-05,2400642
2019-06,1940611
2019-07,1604472
2019-08,1426229
2019-09,1283302


In [9]:
idx_record %>% group_by(year(IndexDate)) %>% summarise(cnt = n()) 

year(IndexDate),cnt
<dbl>,<int>
2016,1192
2017,19575908
2018,9621432
2019,27
